# Image Identification and Classification with Amazon Bedrock, OpenSearch, and OpenCLIP

Build a generative AI-powered vehicle damage assessment application on AWS using Vector Engine for Amazon OpenSearch Serverless, AI21 Labs Foundation Models, and OpenCLIP

References
- https://github.com/mlfoundations/open_clip

## Install OpenCLIP

In [ ]:
# open_clip packages
%pip install open_clip_torch pillow boto3 -Uq

## Install OpenSearch Python Client

In [ ]:
# opensearch python client
%pip install opensearch-py -Uq

In [ ]:
# optionally, restart kernel to update packages
import os

os._exit(00)

In [ ]:
%pip list | grep 'open-clip-torch\|torch\|opensearch-py\|boto3'

In [ ]:
import csv
import json
import os
import random

import open_clip
import torch
from PIL import Image

In [ ]:
print(torch.__version__)

In [ ]:
# list pre-trained CLIP models
open_clip.list_pretrained()

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

device

In [ ]:
# CLIP ViT L/14 model trained with the LAION-2B English subset of LAION-5B using OpenCLIP
# reference: https://huggingface.co/laion/CLIP-ViT-L-14-laion2B-s32B-b82K
model, _, preprocess = open_clip.create_model_and_transforms(
    model_name="ViT-L-14",
    pretrained="laion2b_s32b_b82k",
    device=device,
)

tokenizer = open_clip.get_tokenizer("ViT-L-14")

## Create Embeddings

In [ ]:
# create image embedding using open clip
def create_image_embedding(image_path, device):
    image = preprocess(Image.open(image_path)).unsqueeze(0)
    image = image.to(device)

    # # visualize preprocessed image
    # image_vis = image[0]/2 + 0.5 # unnormalize the image
    # plt.axis("off")
    # plt.imshow(np.transpose(image_vis, (1,2,0))) # convert from tensor to image
    # plt.show()

    with torch.no_grad(), torch.cuda.amp.autocast():
        image_features = model.encode_image(image)
    return image_features.tolist()[0]

In [ ]:
# create text embedding using open clip
def create_text_embedding(text):
    text = tokenizer(text)
    text = text.to(device)

    with torch.no_grad(), torch.cuda.amp.autocast():
        text_features = model.encode_text(text)
    return text_features.tolist()[0]

In [ ]:
# assign a random severity to damaged vehicle image
def random_severity():
    damage_severity_list = ["minor", "moderate", "severe"]
    severity = random.choice(damage_severity_list)
    return severity

In [ ]:
index_name = "open-clip-vehicle-eval-index"

In [ ]:
# create vector embeddings and corresponding opensearch documents for image
def create_documents_with_embeddings_to_csv(document_path, image_directory, damage):
    embedding_request_body = ""
    row_count = 0
    header = ["document"]
    ext = [".png", ".jpeg", ".jpg"]

    with open(document_path, "w", encoding="UTF8", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(header)

        for dir_path, dir_names, filenames in os.walk(image_directory):
            for filename in filenames:
                if filename.lower().endswith(tuple(ext)):
                    description = dir_path.split("/")[-1]
                    file_path = os.path.join(dir_path, filename)
                    try:
                        embedding = create_image_embedding(file_path, device)
                        embedding_request_body = json.dumps(
                            {
                                "image_vector": embedding,
                                "name": filename,
                                "file_path": file_path,
                                "description": description,
                                "severity": random_severity() if damage else "none",
                            }
                        )
                        writer.writerow([embedding_request_body])
                        print(f"Creating document: {row_count}", end="\r")
                        row_count += 1
                    except Exception as ex:
                        print(ex)

In [ ]:
create_documents_with_embeddings_to_csv(
    "embeddings/open_clip_embeddings_undamaged.csv", "undamaged_car_images/", False
)

In [ ]:
create_documents_with_embeddings_to_csv(
    "embeddings/open_clip_embeddings_damaged.csv", "damaged_car_images/", True
)

# Amazon OpenSearch Serverless Vectorsearch Collection

References

- https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-amazon-opensearch-serverless

In [ ]:
# create opensearch serverless client
# https://opensearch.org/docs/latest/clients/python-low-level/#connecting-to-amazon-opensearch-serverless
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3

host = "<your_host>.us-east-1.aoss.amazonaws.com"
region = "us-east-1"
service = "aoss"
credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region, service)

client = OpenSearch(
    hosts=[{"host": host, "port": 443}],
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection,
    pool_maxsize=20,
)

client

In [ ]:
# delete index
try:
    response = client.indices.delete(index_name)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex.error)

In [ ]:
# create new index
index_body = {
    "mappings": {
        "properties": {
            "description": {"type": "text"},
            "file_path": {"type": "text"},
            "image_vector": {
                "type": "knn_vector",
                "dimension": 768,
                "method": {
                    "engine": "nmslib",
                    "space_type": "cosinesimil",
                    "name": "hnsw",
                    "parameters": {"ef_construction": 768, "m": 16},
                },
            },
            "severity": {"type": "text"},
            "name": {"type": "text"},
        }
    },
    "settings": {
        "index": {
            "number_of_shards": 4,
            "knn.algo_param": {"ef_search": 768},
            "knn": True,
        }
    },
}

try:
    response = client.indices.create(index_name, body=index_body)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex)

In [ ]:
# describe new vector index
try:
    response = client.indices.get(index_name)
    print(json.dumps(response, indent=2))
except Exception as ex:
    print(ex.error)

In [ ]:
# load documents from csv file and index to opensearch
# https://opensearch.org/docs/latest/clients/python-high-level/#indexing-a-document
def index_documents(document_path):
    with open(document_path) as f:
        row_count = sum(1 for _ in f) - 1

        f.seek(0)
        reader = csv.reader(f)
        next(reader)
        for index, row in enumerate(reader):
            try:
                document = eval(row[0])
                client.index(
                    index=index_name,
                    body=document,
                )
                print(f"Indexing document: {index+1}/{row_count}", end="\r")
            except Exception as ex:
                print(ex)

In [ ]:
index_documents("embeddings/open_clip_embeddings_undamaged.csv")

In [ ]:
index_documents("embeddings/open_clip_embeddings_damaged.csv")

## Image Search using Text Embedding

In [ ]:
text_query = "photo of a scratched car door"

text_embedding = create_text_embedding(text_query)
print(text_embedding[0:12])
print(len(text_embedding))

In [ ]:
# query index using text embedding
query = {
    "size": 10,
    "_source": {"excludes": ["image_vector"]},
    "query": {
        "knn": {
            "image_vector": {
                "vector": text_embedding,
                "k": 10,
            }
        }
    },
}

try:
    text_based_search_response = client.search(body=query, index=index_name)
except Exception as ex:
    print(ex)

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

x = 1
rows = 4
columns = 5

fig = plt.figure(figsize=(14, 7))

print(f'Text-based query: "{text_query}"\n')

for hit in text_based_search_response["hits"]["hits"]:
    fig.add_subplot(rows, columns, x)
    image = np.array(Image.open(hit["_source"]["file_path"]))
    plt.axis("off")
    plt.title(
        f'{hit["_source"]["description"][0:35]}...\nSeverity: {hit["_source"]["severity"]}\n{hit["_score"]:.2%}',
        fontsize=10,
    )
    plt.imshow(image)
    x += 1

## Image Search using Image Embedding

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

search_image_path_01 = "test_images/undamaged/test_image_09.png"
search_image_01 = np.array(Image.open(search_image_path_01))
plt.box(on=None)
plt.axis("off")
plt.imshow(search_image_01)

In [ ]:
image_embedding_01 = create_image_embedding(search_image_path_01, device)
print(image_embedding_01[0:12])

In [ ]:
# query index using image embedding
# https://docs.aws.amazon.com/opensearch-service/latest/developerguide/knn.html
query = {
    "size": 10,
    "_source": {"excludes": ["image_vector"]},
    "query": {
        "knn": {
            "image_vector": {
                "vector": image_embedding_01,
                "k": 10,
            }
        }
    },
}

try:
    image_based_search_response_01 = client.search(body=query, index=index_name)
except Exception as ex:
    print(ex)

In [ ]:
from matplotlib import pyplot as plt
from PIL import Image
import numpy as np

plt.title("Search image", fontsize=10)

plt.rcParams["figure.figsize"] = [3, 2]
plt.rcParams["figure.autolayout"] = True

plt.box(on=None)
plt.axis("off")
plt.imshow(search_image_01)

index = 1
rows = 4
columns = 5

print(f"Image-based query results\n")

fig = plt.figure(figsize=(18, 9))

for hit in image_based_search_response_01["hits"]["hits"]:
    fig.add_subplot(rows, columns, index)
    image = np.array(Image.open(hit["_source"]["file_path"]))
    plt.axis("off")
    plt.title(
        f'{hit["_source"]["description"][0:35]}...\nSeverity: {hit["_source"]["severity"]}\n{hit["_score"]:.2%}',
        fontsize=11,
    )
    plt.imshow(image)
    index += 1